In [2]:
import random
import numpy as np
import matplotlib.pyplot as plt
import time

# Générer le graphe TSP incomplet
def generate_tsp_incomplete(num_sommets, max_poids_arete):
    matrice = [[None for _ in range(num_sommets)] for _ in range(num_sommets)]
    intervalles = {}
    for i in range(num_sommets):
        intervalle_start = random.randint(0, 10)
        intervalle_end = intervalle_start + random.randint(5, 20)
        intervalles[i] = (intervalle_start, intervalle_end)
    for i in range(num_sommets):
        for j in range(i + 1, num_sommets):
            if random.choice([True, False]):
                poids = random.randint(1, max_poids_arete)
                matrice[i][j] = poids
                matrice[j][i] = poids
    for i in range(num_sommets):
        pas_connexe = True
        for j in range(num_sommets):
            if matrice[i][j] is not None:
                pas_connexe = False
        if pas_connexe:
            poids = random.randint(1, max_poids_arete)
            matrice[i][num_sommets-1] = poids
            matrice[num_sommets-1][i] = poids
    for i in range(num_sommets):
        for j in range(num_sommets):
            if matrice[i][j] is None:
                matrice[i][j] = float('inf')
    return matrice, intervalles

# Générer les données du problème
num_sommets = 5
max_poids_arete = 10
tsp_matrice, tsp_intervalles = generate_tsp_incomplete(num_sommets, max_poids_arete)
print("\nMatrice pondérée:")
for row in tsp_matrice:
    print(row)

# Afficher les intervalles
print("\nIntervalles des sommets")
print(tsp_intervalles)

def calculate_path_length(matrice, solution):
    length = 0
    for i in range(len(solution) - 1):
        length += matrice[solution[i]][solution[i + 1]]
    length += matrice[solution[-1]][solution[0]]  # return to the start point
    return length

def is_valid_solution(solution, matrice, intervalles):
    path_length = calculate_path_length(matrice, solution)
    for city in solution:
        min_weight, max_weight = intervalles[city]
        if not (min_weight <= path_length <= max_weight):
            return False
    return True

def generate_neighbor_solution(current_solution):
    neighbor = current_solution[:]
    i, j = random.sample(range(len(neighbor)), 2)
    neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    return neighbor

def tabu_search(matrice, intervalles, tabu_size=10, num_iterations=1000):
    num_sommets = len(matrice)
    execution_times = []
    distances = []

    # Generate an initial random valid solution
    while True:
        current_solution = list(range(num_sommets))
        random.shuffle(current_solution)
        if is_valid_solution(current_solution, matrice, intervalles):
            break

    current_length = calculate_path_length(matrice, current_solution)
    best_solution = current_solution[:]
    best_length = current_length

    tabu_list = []

    for _ in range(num_iterations):
        start_time = time.time()
        
        # Generate all potential neighbors
        neighbors = []
        for _ in range(100):
            neighbor = generate_neighbor_solution(current_solution)
            if is_valid_solution(neighbor, matrice, intervalles):
                neighbors.append(neighbor)
        
        # Find the best neighbor that is not in the tabu list
        next_solution = None
        next_length = float('inf')
        for neighbor in neighbors:
            neighbor_length = calculate_path_length(matrice, neighbor)
            if neighbor not in tabu_list and neighbor_length < next_length:
                next_solution = neighbor
                next_length = neighbor_length
        
        if next_solution is None:
            break  # No valid move found
        
        # Update the current solution
        current_solution = next_solution
        current_length = next_length
        
        # Update the best solution found so far
        if current_length < best_length:
            best_solution = current_solution
            best_length = current_length
        
        # Add the current solution to the tabu list
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)  # Maintain the tabu list size
        
        end_time = time.time()
        execution_times.append(end_time - start_time)
        distances.append(current_length)
    
    return best_solution, best_length, execution_times, distances

def plot_route(solution, points):
    route = points[solution]
    plt.figure(figsize=(10, 6))
    plt.plot(route[:, 0], route[:, 1], 'o-', label='Route')
    plt.plot([route[-1, 0], route[0, 0]], [route[-1, 1], route[0, 1]], 'o-')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('TSP Route')
    plt.legend()
    plt.show()

def plot_performance(execution_times, distances):
    plt.figure(figsize=(10, 6))
    plt.plot(distances, label='Path Length')
    plt.xlabel('Iteration')
    plt.ylabel('Path Length')
    plt.title('Tabu Search Performance')
    plt.legend()
    plt.show()

points = np.random.rand(num_sommets, 2) * 100

best_solution, best_length, execution_times, distances = tabu_search(tsp_matrice, tsp_intervalles)

print(f"Solution trouvée : {best_solution} avec une longueur de chemin de : {best_length}")

if best_solution:
    plot_route(best_solution, points)

plot_performance(execution_times, distances) 


Matrice pondérée:
[inf, 7, 6, inf, 9]
[7, inf, 9, 2, 7]
[6, 9, inf, 10, inf]
[inf, 2, 10, inf, inf]
[9, 7, inf, inf, inf]

Intervalles des sommets
{0: (10, 16), 1: (1, 14), 2: (10, 15), 3: (0, 6), 4: (1, 19)}


KeyboardInterrupt: 